# ONS Charts

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os

import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import common_core
import ons_core

## Load CSV Files

Load CSV data into cache

In [2]:
def loadCsvFile(areaType, areaName):
    '''Load CSV file into numpy array'''
    csvPath = os.path.join(common_core.projdir, "data", "ons-deaths", "csv", "weekly", "deaths", areaType)
    csvFn = os.path.join(csvPath, common_core.getSafeName(areaName) + ".csv")

    try:
        with open(csvFn, 'r') as f:
            reader = csv.reader(f, delimiter = ',')

            dtype = []
            converters = {}
            colNames = next(reader)

            for i in range(len(colNames)):
                colName = colNames[i]
                if colName == ons_core.WEEK_ENDED:
                    dtype.append((colName, "U10"))
                else:
                    dtype.append((colName, "u4"))
                    converters[i] = lambda s: int(s or 0)

            data = np.genfromtxt(f, dtype=dtype, converters=converters, delimiter=",")

    except:
        print(f"Failed to load CSV data for {areaName}")
        raise
        
    return data


def loadCsvFiles():
    '''Convert weekly registrations into weekly occurrences'''
    
    cache = {}
    
    for nationName in [common_core.ENGLAND_WALES, common_core.WALES]:
        cache[nationName] = loadCsvFile("nation", nationName)

    for regionName in common_core.regionNames:
        cache[regionName] = loadCsvFile("region", regionName)

    return cache

## Plot Data

Simple plots of ONS data

In [3]:
def plotData(cache, areaNames, maxWeeks = 52):
    '''Plot data for visual inspection'''
    
    # week_ended, week_number, total_registrations, covid_registrations, covid_occurrences
    # week_ended, week_number, total_deaths

    fig, ax = plt.subplots(1, figsize=(16, 6))
    
    ewData = cache[common_core.ENGLAND_WALES]

    for areaName in areaNames:
        areaData = cache[areaName]

        ax.set_title(f"Weekly Deaths in {areaName}")

        ax.set_ylabel('Number of deaths')

        y_points = areaData[ons_core.TOTAL_REGISTRATIONS][-maxWeeks:]
        x_points = np.arange(len(y_points))       
        ax.plot(x_points, y_points, label = "Total Registrations", color='navy', linestyle="dotted")

        y_points = areaData[ons_core.TOTAL_OCCURRENCES][-maxWeeks:]
        x_points = np.arange(len(y_points))       
        ax.plot(x_points, y_points, label = "Total Occurrences", color='navy')

        y_points = areaData[ons_core.COVID_REGISTRATIONS][-maxWeeks:]
        x_points = np.arange(len(y_points))       
        ax.plot(x_points, y_points, label = "COVID-19 Registrations", color='red', linestyle="dotted")

        y_points = areaData[ons_core.COVID_OCCURRENCES][-maxWeeks:]
        x_points = np.arange(len(y_points))       
        ax.plot(x_points, y_points, label = "COVID-19 Occurrences", color='red')

        x_ticks = np.array(areaData[ons_core.WEEK_ENDED])[-maxWeeks:]
        ax.set_xticks(np.arange(0, len(x_ticks)))
        ax.set_xticklabels(x_ticks, rotation=90)

        ax.set_yticks(np.arange(0, 24000, 1000))
        ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))
        
        ax.legend()

## Interactive Testing

In [4]:
if __name__ == '__main__':
    # Cache can either be re-used or loaded from CSV files
    cache = loadCsvFiles()

    # Simple charts for review
    areaNames = [common_core.ENGLAND_WALES]
    #plotData(cache, areaNames)